In [1]:
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from PIL import Image
import numpy as np

In [2]:
from PIL import Image
import numpy as np

img = Image.open('preprocessed_data/test/h/2.jpg').convert("L")  # chuyển ảnh sang grayscale
threshold = 128
binary_img = img.point(lambda x: 0 if x < threshold else 255, '1')  # ảnh nhị phân
binary_img.show()

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\kubao\\TOAN\\VIETNAMESE-CHARACTER-RECOGNITION\\DoAn\\HTN\\preprocessed_data\\test\\h\\2.jpg'

In [3]:
# Cấu hình
image_size = (224, 224)  # ResNet yêu cầu 224x224
batch_size = 128
num_epochs = 10
learning_rate = 0.001

# Thiết bị
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [4]:
class DebugImageFolder(datasets.ImageFolder):
    def __getitem__(self, index):
        path, target = self.samples[index]
        img = Image.open(path)
        if self.transform is not None:
            img = self.transform(img)
        return img, target

In [9]:
# Tiền xử lý ảnh
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize(image_size),
    transforms.ToTensor(),
])

# Dataset
train_dataset = DebugImageFolder('../anhmoi', transform=transform)
test_dataset = DebugImageFolder('preprocessed_data/test', transform=transform)

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Số lớp
num_classes = len(train_dataset.classes)


In [6]:
# Load mô hình ResNet18 pretrained
model = models.resnet18(pretrained=True)

# Thay thế lớp fully connected để phù hợp với số lớp của bạn
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)


c:\Users\admin\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\admin\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\admin/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:37<00:00, 1.25MB/s]


In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


In [11]:
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    print(f"Epoch {epoch+1}: Loss = {running_loss/len(train_loader):.4f}, Acc = {100*correct/total:.2f}%")


Epoch 1/10: 100%|██████████| 19/19 [03:21<00:00, 10.62s/it]


Epoch 1: Loss = 0.7367, Acc = 80.52%


Epoch 2/10: 100%|██████████| 19/19 [08:38<00:00, 27.31s/it]


Epoch 2: Loss = 0.1119, Acc = 97.63%


Epoch 3/10: 100%|██████████| 19/19 [08:41<00:00, 27.43s/it]


...

Epoch 8: Loss = 0.0035, Acc = 99.91%


Epoch 9/10: 100%|██████████| 19/19 [02:38<00:00,  8.36s/it]


Epoch 9: Loss = 0.0350, Acc = 99.31%


Epoch 10/10: 100%|██████████| 19/19 [02:34<00:00,  8.12s/it]

Epoch 10: Loss = 0.0483, Acc = 98.58%


In [12]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in tqdm(test_loader, desc="Đang đánh giá (test)", leave=False):
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"\n✅ Độ chính xác trên tập test: {100 * correct / total:.2f}%")



✅ Độ chính xác trên tập test: 45.71%


In [13]:
model_path = "resnet18_trained.pth"
torch.save(model.state_dict(), model_path)
print(f"✅ Đã lưu mô hình tại: {model_path}")

✅ Đã lưu mô hình tại: resnet18_trained.pth


In [14]:
# Khởi tạo lại mô hình ResNet
model = models.resnet18(pretrained=False)  # không cần pretrained nữa
model.fc = nn.Linear(model.fc.in_features, num_classes)
model.load_state_dict(torch.load("resnet18_trained.pth"))
model = model.to(device)
model.eval()

c:\Users\admin\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\admin\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
C:\Users\admin\AppData\Local\Temp\ipykernel_26864\3455384497.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flippe

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  